In [1]:
!pip install transformers -q
!pip install wandb -q
!pip install rouge-score -q

  Preparing metadata (setup.py) ... done


In [2]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, MT5ForConditionalGeneration
from rouge_score import rouge_scorer

# WandB – Import the wandb library
import wandb

# !nvidia-smi
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [3]:
# Login to wandb to log the model run and all the parameters
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import T5Tokenizer, MT5ForConditionalGeneration
from torch.optim.lr_scheduler import ReduceLROnPlateau
from rouge_score import rouge_scorer
import wandb

class CustomDataset(Dataset):
    """
    Custom dataset class for handling Telugu text data.
    Prepares data for both original stories and generated summaries.
    """
    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.summary = self.data.summary
        self.story = self.data.story

    def __len__(self):
        return len(self.summary)

    def __getitem__(self, index):
        story = str(self.story[index])
        story = ' '.join(story.split())
        summary = str(self.summary[index])
        summary = ' '.join(summary.split())

        source = self.tokenizer.batch_encode_plus(
            [story],
            max_length=self.source_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        target = self.tokenizer.batch_encode_plus(
            [summary],
            max_length=self.summ_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long),
            'source_mask': source_mask.to(dtype=torch.long),
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

def train(epoch, tokenizer, model, device, loader, optimizer):
    """
    Training function that performs training on the give dataset. It also includes loss calculation and optimization.
    """
    model.train()
    total_loss = 0

    for _, data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        labels = y[:, 1:].clone().detach()
        labels[y[:, 1:] == tokenizer.pad_token_id] = -100

        ids = data['source_ids'].to(device, dtype=torch.long)
        mask = data['source_mask'].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=labels
        )

        loss = outputs[0]
        total_loss += loss.item()

        if _ % 10 == 0:
            wandb.log({"Training Loss": loss.item()})

        if _ % 500 == 0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

    return total_loss / len(loader)

def validate(epoch, tokenizer, model, device, loader):
    """
    Validation function to generate summaries and calculate metrics.
    Returns predictions and actual summaries for comparison.
    """
    model.eval()
    predictions = []
    actuals = []
    total_loss = 0

    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype=torch.long)
            ids = data['source_ids'].to(device, dtype=torch.long)
            mask = data['source_mask'].to(device, dtype=torch.long)

            generated_ids = model.generate(
                input_ids=ids,
                attention_mask=mask,
                max_length=400,
                min_length=200,
                do_sample=True,
                num_beams=8,
                no_repeat_ngram_size=4,
                temperature=0.7,
                length_penalty=2.0,
                early_stopping=False,
                top_k=50,
                top_p=0.9,
                repetition_penalty=1.8,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id
            )

            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True) for t in y]

            predictions.extend(preds)
            actuals.extend(target)

            if _ % 100 == 0:
                print(f'Completed {_}')

    return predictions, actuals

def main():
    # Initialize wandb
    wandb.init(project="transformers_tutorials_summarization")

    # Configuration
    config = wandb.config
    config.TRAIN_BATCH_SIZE = 4
    config.VALID_BATCH_SIZE = 4
    config.TRAIN_EPOCHS = 30
    config.VAL_EPOCHS = 1
    config.LEARNING_RATE = 2e-4
    config.SEED = 42
    config.MAX_LEN = 512
    config.SUMMARY_LEN = 150

    # Set random seeds
    torch.manual_seed(config.SEED)
    np.random.seed(config.SEED)
    torch.backends.cudnn.deterministic = True

    # Initialize tokenizer and model
    tokenizer = T5Tokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
    model = MT5ForConditionalGeneration.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)

    # Load and preprocess data
    df = pd.read_excel('telugu_stories.xlsx')
    df = df[['summary', 'story']]
    df.story = 'summarize: ' + df.story
    print(df.head())

    # Split data
    train_size = 0.8
    train_dataset = df.sample(frac=train_size, random_state=config.SEED)
    val_dataset = df.drop(train_dataset.index).reset_index(drop=True)
    train_dataset = train_dataset.reset_index(drop=True)

    print(f"FULL Dataset: {df.shape}")
    print(f"TRAIN Dataset: {train_dataset.shape}")
    print(f"TEST Dataset: {val_dataset.shape}")

    # Create datasets
    training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)
    val_set = CustomDataset(val_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

    # Create dataloaders
    train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 2
    }

    val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 2
    }

    training_loader = DataLoader(training_set, **train_params)
    val_loader = DataLoader(val_set, **val_params)

    # Initialize optimizer and scheduler
    optimizer = torch.optim.Adam(params=model.parameters(), lr=config.LEARNING_RATE)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

    # Log metrics with wandb
    wandb.watch(model, log="all")

    # Training loop
    print('Starting Fine-Tuning')
    best_valid_loss = float('inf')

    for epoch in range(config.TRAIN_EPOCHS):
        avg_loss = train(epoch, tokenizer, model, device, training_loader, optimizer)
        scheduler.step(avg_loss)

        # # Validating every 5 epochs
        if epoch % 5 == 0:
            predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)

            # Save the model if validation loss is improved
            if avg_loss < best_valid_loss:
                best_valid_loss = avg_loss
                model.save_pretrained('./best_model')
                tokenizer.save_pretrained('./best_model')
                print(f"Saved best model at epoch {epoch}")

    # Final validation and save predictions
    print('Generating final summaries')
    predictions, actuals = validate(0, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text': predictions, 'Actual Text': actuals})
    final_df.to_csv('predictions.csv')
    print('Output files generated')

    # Save the final model
    output_dir = "./fine_tuned_mt5_telugu"
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"Final model saved to: {output_dir}")

if __name__ == '__main__':
    main()

wandb: Currently logged in as: arsitha-lbf315 (arsitha-lbf315-george-mason-university). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

...

                                             summary  \
0  పేద కుటుంబం నుండి వచ్చిన రాజేష్, తన విద్య కోసం...   
1  వృద్ధుడైన శాస్త్రి తన విలువైన తోటను కాపాడటం కో...   
2  ఆకాశంలోని మబ్బులను ప్రేమించే చిన్నారి రవి జీవి...   
3  చీన దేవగిరి కోటలో నివసించే ఒక వృద్ధ శిల్పి జీవ...   
4  రాము అనే బాలుడు చిన్న పల్లెటూరిలో పుట్టి పెరిగ...   

                                               story  
0  summarize: రాజేష్ అనే కుర్రవాడు చిన్న పల్లెటూర...  
1  summarize: శాస్త్రి అనే వృద్ధుడు తన వంట చెట్ల ...  
2  summarize: వసంత పల్లెలో ఒక చిన్న పిల్లవాడు రవి...  
3  summarize: దేవగిరి అనే ప్రాచీన కోటలో ఒక వృద్ధ ...  
4  summarize: రాము అనే చిన్న బాలుడు తన తల్లిదండ్ర...  
FULL Dataset: (200, 2)
TRAIN Dataset: (160, 2)
TEST Dataset: (40, 2)
Starting Fine-Tuning


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch: 0, Loss: 5.697854995727539
Completed 0


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 84, 'num_beams': 4, 'length_penalty': 0.6, 'no_repeat_ngram_size': 2}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Saved best model at epoch 0
Epoch: 1, Loss: 1.9551866054534912
Epoch: 2, Loss: 1.6142200231552124
Epoch: 3, Loss: 1.4035757780075073
Epoch: 4, Loss: 1.3729761838912964
Epoch: 5, Loss: 0.9112350344657898
Completed 0
Saved best model at epoch 5
Epoch: 6, Loss: 0.796274721622467
Epoch: 7, Loss: 0.6849696040153503
Epoch: 8, Loss: 0.5307756662368774
Epoch: 9, Loss: 0.588497519493103
Epoch: 10, Loss: 0.43642503023147583
Completed 0
Saved best model at epoch 10
Epoch: 11, Loss: 0.34087103605270386
Epoch: 12, Loss: 0.28361260890960693
Epoch: 13, Loss: 0.3387318551540375
Epoch: 14, Loss: 0.3234672546386719
Epoch: 15, Loss: 0.22598335146903992
Completed 0
Saved best model at epoch 15
Epoch: 16, Loss: 0.246625617146492
Epoch: 17, Loss: 0.15996159613132477
Epoch: 18, Loss: 0.1998007446527481
Epoch: 19, Loss: 0.1707136332988739
Epoch: 20, Loss: 0.1715845912694931
Completed 0
Saved best model at epoch 20
Epoch: 21, Loss: 0.10438496619462967
Epoch: 22, Loss: 0.14618141949176788
Epoch: 23, Loss: 0.117

In [54]:
from transformers import MT5ForConditionalGeneration, T5Tokenizer
import torch

def generate_summary(text, model_path="./fine_tuned_mt5_telugu"):
    """
    Generate summary for a given Telugu text using the trained model.
    that takes Args as text: Input Telugu text, model_path: Path to the saved model
    and returns generated summary in Telugu.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    tokenizer = T5Tokenizer.from_pretrained(model_path)
    model = MT5ForConditionalGeneration.from_pretrained(model_path).to(device)
    model.eval()

    inputs = tokenizer(f"summarize: {text}", return_tensors="pt", max_length=512, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        summary_ids = model.generate(
            inputs["input_ids"],
            max_length=400,
            min_length=200,
            num_beams=8,
            do_sample=True,
            temperature=0.7,
            length_penalty=2.0,
            no_repeat_ngram_size=4,
            early_stopping=False,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.8,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

def main():
    print("Welcome to Telugu Story Summarizer!")
    print("Enter your story in Telugu and get a summary.")
    print("Type 'quit' to exit the program.")
    print("-" * 50)

    while True:
        try:
            print("\nEnter your Telugu story (or 'quit' to exit):")
            story = input()

            if story.lower() == 'quit':
                print("Thank you for using Telugu Story Summarizer!")
                break

            if not story.strip():
                print("Please enter a valid story!")
                continue

            print("\nGenerating Abstarctive summary...")
            summary = generate_summary(story)
            print("\nGenerated Abstractie Summary:")
            print("-" * 20)
            print(summary)
            print("-" * 50)

        except Exception as e:
            print(f"\nError occurred: {str(e)}")
            print("Please try again with a different story.")

if __name__ == "__main__":
    main()

Welcome to Telugu Story Summarizer!
Enter your story in Telugu and get a summary.
Type 'quit' to exit the program.
--------------------------------------------------

Enter your Telugu story (or 'quit' to exit):
అనగనగా, ఒక పెద్ద అడవిలో రామచిలుక అనే తెలివైన పక్షి నివసించేది. ఇది ఎల్లప్పుడూ ఇతర పక్షులకు సహాయం చేస్తూ, స్నేహపూర్వకంగా ఉండేది. అడవిలోని ప్రతి జంతువుకు ఇది ఎంతో ఇష్టమైన పక్షి. రామచిలుక తన తెలివితేటలతో సమస్యలను పరిష్కరించడంలో అందరికీ ఆదర్శంగా నిలిచేది. ఒకసారి, ఆ అడవిని ఒక పెద్ద ఎడారి తాకింది. ఎడారి ప్రభావం అడవిని నాశనం చేయడం ప్రారంభించింది. చెట్లు వాడిపోవడం, నీటి వనరులు ఎండిపోవడం, జంతువుల ఆహారం అందకపోవడం వంటి సమస్యలు కలిగాయి. అడవిలోని జంతువులన్నీ ఈ పరిస్థితిని ఎదుర్కోలేక అల్లాడిపోతున్నాయి. రామచిలుక ఈ పరిస్థితిని గమనించి, "నేను ఒక మార్గం కనుగొని, ఈ సమస్యను పరిష్కరిస్తాను," అని చెప్పింది. అది తన ప్రయాణాన్ని ప్రారంభించింది. దూరంగా ఉన్న పచ్చని ప్రాంతాలను కనుగొనడానికి అటూ ఇటూ ఎగిరింది. పక్షి అయినందున, అది ఆకాశం నుండి దూర ప్రాంతాలను పరిశీలించగలిగింది. కొన్ని రోజుల ప్రయాణం తర్వాత, రామచ

In [51]:
#moderate-length story
!pip install rouge
from rouge import Rouge
Generated_summary = "ఆ సరస్సులో గంగమ్మ అనే తెలివైన చేప రాణిగా జీవించేది. ఇది తన గుంపు చేపలందరినీ ప్రేమగా, జాగ్రత్తగా చూసుకునేది. ఒక రోజు, మత్స్యకారులు పెద్ద వలతో సరస్సులో చేపలను పట్టుకోవడానికి వచ్చారు. గంగమ్మ మత్ద్కారుల కదలికలను గమనించి, తన చేపల గుంపును రక్షించడానికి ఒక వ్యూహాన్ని ఆలోచించింది. ఇతర చేపలను పచ్చని మొక్కల మధ్య దాక్కోవాలని, సరస్సు లోతైన భాగాల్లో తలదాచుకోవాలని చెప్పింది. చేపలన్నీ గంగమ్మ సూచనను పాటించాయి. వలలో చిక్కుకున్న కొద్ది మంది చేపలను గంగమ్మ గుంపును కాపాడాయి. ఈ సంఘటన తర్వాత, సరస్సు అన్ని చేపలకూ సురక్షిత ప్రదేశంగా మారింది. చేపలంతా గంగమ్మను మెచ్చుకున్నాయి."
Actual_summary = "తక్కువ లోతైన సరస్సులో గంగమ్మ అనే తెలివైన చేప రాణిగా జీవించేది. ఇది తన గుంపు చేపలందరినీ ప్రేమగా, జాగ్రత్తగా చూసుకునేది. ఒక రోజు, మత్స్యకారులు పెద్ద వలతో సరస్సులో చేపలను పట్టుకోవడానికి వచ్చారు. గంగమ్మ మత్స్యకారుల కదలికలను గమనించి, తన చేపల గుంపును రక్షించడానికి ఒక వ్యూహాన్ని ఆలోచించింది.ఇతర చేపలను పచ్చని మొక్కల మధ్య దాక్కోవాలని, సరస్సు లోతైన భాగాల్లో తలదాచుకోవాలని చెప్పింది. చేపలన్నీ గంగమ్మ సూచనను పాటించాయి. వలలో చిక్కిన కొద్దిమందిని కూడా గంగమ్మ తన గుంపుతో కలిసి బయటికి తీసుకొచ్చింది. మత్స్యకారులు వల విసిరినా పెద్దగా చేపలు చిక్కకపోవడంతో అవస్థపడి సరస్సును వదిలి వెళ్లిపోయారు.తరువాత, సరస్సులోని చేపలన్నీ గంగమ్మను తమ రక్షకురాలిగా భావించి మరింత గౌరవించాయి. ఇతర ప్రాంతాల నుండి వచ్చిన చేపలు కూడా గంగమ్మ వద్ద సహాయం పొందుతూ, తన తెలివితేటలను మెచ్చుకున్నాయి. గంగమ్మ తన గుంపుకు ప్రేమతో మాత్రమే కాకుండా, జాగ్రత్తలతో కూడా రాణిగా నిలిచింది.ఆ సరస్సు అన్ని చేపలకూ సురక్షిత ప్రదేశంగా మారింది. గంగమ్మ కథ చేపల గుంపు మధ్య మాత్రమే కాకుండా, ఇతర ప్రాంతాల మత్స్యకుల కథల్లోనూ ప్రసిద్ధిగా నిలిచింది. గంగమ్మ తన తెలివి, ధైర్యంతో, మరియు నిరంతరం ప్రేమతో సముదాయానికి ఒక స్ఫూర్తిదాయక నేతగా నిలిచింది. ఈ కథ పాత తరాలపాటు చేపల గుంపుల్లో చెరగని గుర్తుగా నిలిచింది."
rouge = Rouge()
scores = rouge.get_scores(Generated_summary ,Actual_summary)
print(scores)

[{'rouge-1': {'r': 0.49056603773584906, 'p': 0.8666666666666667, 'f': 0.6265060194803311}, 'rouge-2': {'r': 0.37681159420289856, 'p': 0.7536231884057971, 'f': 0.5024154544927536}, 'rouge-l': {'r': 0.49056603773584906, 'p': 0.8666666666666667, 'f': 0.6265060194803311}}]


In [52]:
#short story
!pip install rouge
from rouge import Rouge
Generated_summary = "బబుజ్జి ఒక తోటలో పడి ఉన్న పూవులను చూసి ఆనందంగా గడిపాడు. ఆ పూవులను జాగ్రత్తగా ఏరుకుని, తన అమ్మకు అందించేందుకు ఒక అందమైన పూలగుచ్చం కట్టాడు. ఆ గుచ్చం సువాసనతో కాదు, తన ముద్దు మనసుతో కూడా నిండిపోయింది. బుజ్జి ప్రతి ఉదయం తన తల్లికి పూలతో అనుభూతిని ఇవ్వడం ప్రారంభించాడు. ఆ రోజు నుండి బుజ్జి తన స్నేహితులకు కొత్త అనుభూతిని ఇచ్చాడు. పూలు పంచుకున్న తల్లి బుజ్జికి కొత్త ఆశాజ్యోతులుగా నిలిచాడు. ఈ అనుభవం తర్వాత బుజ్జి బాలుడి జీవితం మళ్ళీ పూలంతో కళకళలాడింది. పిల్లలతో ఆడుకుంటూ బుజ్జి కష్టపడితే ఏడాది చివర్లో పూలంతో ఆనందంగా జీవించడం ప్రారంభించాడు. అతను చిన్న బాలుడి నుండి పూల ఆశీర్వాదం పొందాడు."
Actual_summary = "బుజ్జి అనే చిన్న బాలుడు తన తోటలో పడి ఉన్న పూలను జాగ్రత్తగా ఏరుకుని, వాటిని ఒక అందమైన పూలగుచ్చంగా మార్చాడు. తల్లికి తన ప్రేమను వ్యక్తం చేయడంలో ఈ పూలగుచ్చం ఒక మంచి సాధనమైంది. తల్లి బుజ్జి చేసిన పనిని చూసి ఎంతో ఆనందించగా, బుజ్జి తల్లి ముఖంలో సంతోషాన్ని చూడటం తనకు పెద్ద ఆనందంగా అనిపించింది.ఈ సంఘటనతో బుజ్జి ప్రతిరోజూ తోటలోకి వెళ్లి కొత్త పూలను ఏరుకొని, తన తల్లికి అందించడాన్ని అలవాటుగా మార్చుకున్నాడు. బుజ్జి తల్లి పూల సువాసనలో కంటే, బుజ్జి మనసులో ఉన్న ప్రేమను మరింత ప్రశంసించింది. తల్లి, తన కుమారుడి ఈ చిన్న కానుకలో అనేక భావోద్వేగాలను చూసి మురిసిపోయింది.ఈ చిన్న పూలగుచ్చం తల్లికుమారుల అనుబంధంలో మరింత గాఢతను తెచ్చింది. బుజ్జి ప్రేమతో చేసిన ప్రతి పూలగుచ్చం, తల్లికి ప్రతిరోజూ కొత్త ఆనందాన్ని ఇచ్చింది. తోటలో పూసే ప్రతి పువ్వు ఈ బంధానికి జీవం పోసింది."
rouge = Rouge()
scores = rouge.get_scores(Generated_summary ,Actual_summary)
print(scores)

[{'rouge-1': {'r': 0.2638888888888889, 'p': 0.30158730158730157, 'f': 0.28148147650370375}, 'rouge-2': {'r': 0.05, 'p': 0.06329113924050633, 'f': 0.05586591685652802}, 'rouge-l': {'r': 0.2638888888888889, 'p': 0.30158730158730157, 'f': 0.28148147650370375}}]
